In [1]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import csv

In [3]:
sc = SparkContext()
spark = SparkSession.builder.appName('Experiment1').getOrCreate()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-2-19aa566e8b2f>:1 

In [10]:
userLibraryRdd = sc.textFile("Datasets/users_libraries.txt")
userLibraryRdd = userLibraryRdd.map(lambda line: line.split(";")).map(lambda line: (line[0],list(map(int,line[1].split(",")))))

In [11]:
def processPaperCsv(line):
    paperCsv = csv.reader([line.replace("\0", "")], delimiter=',', quoting=csv.QUOTE_MINIMAL)
    paperCsvList = next(paperCsv)
    return paperCsvList[0], paperCsvList[14]

paperCsvRdd = sc.textFile("Datasets/papers.csv")
paperCsvRdd = paperCsvRdd.map(processPaperCsv).filter(lambda x: x[1] != "").map(lambda x: (int(x[0]),x[1].split(" ")))

In [12]:
stopWords = sc.textFile("Datasets/stopwords_en.txt")
stopWordsBroadcast = sc.broadcast(stopWords.collect())

In [45]:
def removeStopWords(x):
    abstractwordsList = x.copy()
    for a in abstractwordsList:
        if a in stopWordsBroadcast.value:
            abstractwordsList.remove(a)
    return abstractwordsList

userLibraryJoinPaperRdd = userLibraryRdd.flatMapValues(lambda x: x).map(lambda x: (x[1],x[0])).join(paperCsvRdd)
userLibraryJoinPaperRdd = userLibraryJoinPaperRdd.map(lambda x: (x[1][0],x[1][1]))
groupedByKeyRdd = userLibraryJoinPaperRdd.flatMapValues(lambda x:x).groupByKey().mapValues(list)
userLibraryJoinPaperWithoutStopWordsRdd = groupedByKeyRdd.mapValues(removeStopWords)

In [38]:
userLibraryJoinPaperWithoutStopWordsRdd.take(2)

[('e5cac31dd5df804dd773d8774407f3b1',
  ['developed',
   '{netpath}',
   'resource',
   'curated',
   'human',
   'signaling',
   'pathways.',
   'initial',
   'step,',
   '{netpath}',
   'detailed',
   'maps',
   'number',
   'immune',
   'signaling',
   'pathways,',
   'include',
   'approximately',
   '1,600',
   'reactions',
   'annotated',
   'literature',
   '2,800',
   'instances',
   'transcriptionally',
   'regulated',
   'genes',
   '-',
   'linked',
   '5,500',
   'published',
   'articles.',
   'anticipate',
   '{netpath}',
   'consolidated',
   'resource',
   'human',
   'signaling',
   'pathways',
   'enable',
   'systems',
   'biology',
   'approaches.',
   'kyoto',
   'encyclopedia',
   'genes',
   'genomes',
   '({kegg})',
   'knowledge',
   'base',
   'systematic',
   'analysis',
   'gene',
   'functions',
   'terms',
   'networks',
   'genes',
   'molecules.',
   'major',
   'component',
   '{kegg}',
   '{pathway}',
   'database',
   'consists',
   'graphical',
   'd


KeyboardInterrupt

